# Problem Description

A company wants to set up a new distribution network.
- There are $N$ possible locations for a distribution center (DC).
- Fixed costs per year for opening a distribution center at location $i$ are $F_i$.
- Each DC can ship at most $S_i$ pallets.
- The total demand per year (in pallets) for each delivery address (DA) $j$ is $D_j$.
- The transport costs for one pallet between the DCs and delivery addresses are $c_{ij}$

Objective:
which DCs will be opened such that the total costs are minimized?
Given:
- $N=37$
- Possible locations for DCs is the same as set of DAs
- $S_i=4000$ for all $i$
- $D_j=100$ for all locations $j$
- $F_i=100000$ for all locations $i$
- $c_{ij}$ is equal to the distance between locations (1 per km per pallet) for all locations $i$ and $j$.

Possible locations and distances between location are given in the csv file: *locations.csv*.

Solve the problem:
1. Given $F_i=100000$ for every location $i$
2. Given $F_i=150000$ for every location $i$

# Optimization Model

## Variables

Based on the problem description, we define the following variables:
- The company has $N={1,...,n}$ possible locations and $M={1,...,m}$ addresses.
- For each location $i\in N$, $F_i$ is the fixed cost per year and $S_i$ is the maximum pallet shipped from the location.
- For each address $j\in M$, $D_j$ is the total demand of the address.
- $c_{ij}$ is the cost of sending 1 pallet from location $i$ to address $j$.

## Decision Variable

The decision to open DC in location $i$ is given by $y_i$, a binary variable:
- $y_i=1$ if a DC is opened in location $i$
- $y_i=0$ otherwise
Additionally, the number of pallet sent from DC in location $i$ to address $j$ is given by $x_{ij}$. Therefore, we have $i+ij$ decision variables.

## Constraints

First, demand for each address $j$ should be satisfied through the constraints:
- $\sum^{n}_{i=1}{x_{ij}}=D_j$

For each distribution center $i$, the number of pallet sent shall not exceed maximum pallet shipped $S_i$ and also depends whether a distribution center is opened ($y_i$). Thus, for each location $i$, we have the following constraints:
- $\sum^{m}_{j=1}{x_{ij}}\leq S_iy_i$

## Objective Function

The yearly total fixed and transportation cost of a feasible decision vector $(x,y)$ is given by $f(x,y)$, where
- $f(x,y)=\sum^{n}_{i=1}{\sum^{m}_{j=1}{x_{ij}c_{ij}}}+\sum^{n}_{i=1}{F_iy_i}$

## Formulation

minimize $f(x,y)=\sum^{n}_{i=1}{\sum^{m}_{j=1}{x_{ij}c_{ij}}}+\sum^{n}_{i=1}{F_iy_i}$

subject to
- $\sum^{n}_{i=1}{x_{ij}}=D_j, 1 \leq j \leq m$
- $\sum^{m}_{j=1}{x_{ij}}\leq S_iy_i, 1 \leq i \leq n$
- $x_{ij}\geq 0, 1 \leq i \leq n, 1 \leq j \leq m$
- $y_i\in\{0,1\}$

# Solution

In [2]:
# import all required package

import numpy as np
import pandas as pd
import pulp  # linear programming package for python

In [3]:
# The pallet costs are given in the csv file
facility_location_distances = pd.read_csv('locations.csv')
facility_location_distances = facility_location_distances.set_index('Distance (km)')

In [4]:
facility_location_distances.head()

,Amsterdam,Antwerp,Athens,Barcelona,Berlin,Bern,Brussels,Calais,Cologne,Copenhagen,...,Prague,Rome,Rotterdam,Strasbourg,Stuttgart,The Hague,Turin,Venice,Vienna,Zurich
Distance (km),,,,,,,,,,,,,,,,,,,,,
Amsterdam,0.0,160.0,3082.0,1639.0,649.0,875.0,209.0,385.0,280.0,904.0,...,973.0,1835.0,80.0,683.0,703.0,56.0,1264.0,1449.0,1196.0,861.0
Antwerp,160.0,0.0,2766.0,1465.0,723.0,704.0,46.0,211.0,237.0,861.0,...,870.0,1660.0,100.0,544.0,659.0,139.0,1090.0,1275.0,1180.0,687.0
Athens,3082.0,2766.0,0.0,3312.0,2552.0,2627.0,3021.0,2976.0,2562.0,3414.0,...,2198.0,2551.0,2826.0,2581.0,2428.0,3061.0,2250.0,1995.0,1886.0,2449.0
Barcelona,1639.0,1465.0,3312.0,0.0,1899.0,913.0,1419.0,1399.0,1539.0,2230.0,...,1679.0,1471.0,1565.0,1072.0,1263.0,1589.0,892.0,1327.0,1989.0,1036.0
Berlin,649.0,723.0,2552.0,1899.0,0.0,986.0,782.0,936.0,575.0,743.0,...,354.0,1573.0,697.0,801.0,636.0,712.0,1172.0,1108.0,666.0,863.0


In [5]:
# initialise variables
max_supply_per_city = 4000  # S
demand_per_city = 100  # D
cities = facility_location_distances.columns.values
# get list of locations and distances to calculate costs
number_of_cities = len(cities)
max_supply = dict(zip(cities, [max_supply_per_city] * len(cities)))
demand = dict(zip(cities, [demand_per_city] * len(cities)))

In [6]:
def facility_location_solver(facility_location_distances, S, D, F):
    """
    function to solve capacitated facility problem given input:
        - facility_location_distances <dataframe>: dataframe of every location distances
        - S <dict>: maximum number pallets from each distribution center (DC)
        - D <dict>: number of pallets required from each delivery address (DA)
        - F <dict>: yearly fixed cost for each DC

        output: print out of status, total cost, and DC locations
    """
    cities = facility_location_distances.columns.values

    # decision variables
    # binary variables to open a DC in a city
    open_facility = pulp.LpVariable.dicts("open facility",
                                          (i for i in cities),
                                          cat='Binary')

    # integer variables of number of pallets from each DC to every DA
    num_pallets = pulp.LpVariable.dicts("num pallets",
                                        ((i, j)
                                         for i in cities for j in cities),
                                        lowBound=0,
                                        upBound=None,
                                        cat='Integer')

    # initialise model
    facility_location_problem = pulp.LpProblem(
        "Facility Location Problem", pulp.LpMinimize)

    # objective function
    facility_location_problem += pulp.lpSum(
        # transportation cost
        [num_pallets[i, j] * facility_location_distances.loc[i, j]
            for i in cities for j in cities]

        # fixed cost
        + [open_facility[i] * F[i] for i in cities]
    ), "Total Cost"

    # demand constraint
    for j in cities:
        facility_location_problem += pulp.lpSum(
            num_pallets[i, j] for i in cities) == D[j], "Demand " + j

    # supply constraint
    for i in cities:
        facility_location_problem += pulp.lpSum(
            num_pallets[i, j] for j in cities) <= open_facility[i] * S[i], "Supply " + i

    facility_location_problem.solve()
    print("Status: " + pulp.LpStatus[facility_location_problem.status])
    print("Total cost: " + str(pulp.value(facility_location_problem.objective)))
    for v in facility_location_problem.variables():
        if 'open_facility' in v.name and v.varValue > 0:
            print(v.name.replace('open_facility_', '') + ": " + str(v.varValue))

    return facility_location_problem

In [7]:
# solve for F = 100000
fixed_cost_per_dc = 100000  # F
fixed_cost = dict(zip(cities, [fixed_cost_per_dc] * len(cities)))
facility_problem_f100k = facility_location_solver(
    facility_location_distances, max_supply, demand, fixed_cost)

Status: Optimal
Total cost: 1684800.0
Athens: 1.0
Brussels: 1.0
Hamburg: 1.0
Madrid: 1.0
Munich: 1.0
Rome: 1.0
Turin: 1.0


In [8]:
# solve for F = 150000
fixed_cost_per_dc2 = 150000
fixed_cost2 = dict(zip(cities, [fixed_cost_per_dc2] * len(cities)))
facility_problem_f150k = facility_location_solver(
    facility_location_distances, max_supply, demand, fixed_cost2)

Status: Optimal
Total cost: 2019100.0
Antwerp: 1.0
Athens: 1.0
Madrid: 1.0
Munich: 1.0
Turin: 1.0


# Conclusion

- Given $F_i=100000$ for every location $i$, the optimal solution is to open 7 DCs in the following cities: Athens, Brussels, Hamburg, Madrid, Munich, Rome, and Turin.
- Given $F_i=150000$ for every location $i$, the optimal solution is to open 5 DCs in the following cities: Antwerp, Athens, Madrid, Munich, and Turin.